In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("quiz_07.ipynb")

# Quiz 7

## Instructions

1. ***Do not change any file names or relative locations***
1. After you answer a question, you should run its public tests.
1. After you answer every question, you should:
    1. Restart your kernel and clear all output
    1. Run up to the last cell
    1. Save your notebook
    1. Run the last cell to create a .zip file for Gradescope
    1. Upload this .zip file to Gradescope
    1. ***Make sure your Gradescope autograder results match your local autograder results***
1. This quiz has public and hidden tests:
    1. Public tests check your answers for correct types and shapes but may not completely check your answers
    1. Hidden tests completely check your answers but will not be available until after the due date
    1. *If all tests were public, you could quickly reverse-engineer incorrect answers that trick the autograder*
1. You may ask technical questions on Canvas Discussions, but this quiz is an individual effort.

## Packages and Settings

***The following code cell imports all the packages you need for this quiz.***
***Do not import any other packages.***

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as sco

## Data

I used the following code to download and save the data you need for this quiz.
***You should review the following code but not run it.***

In [3]:
# import yfinance as yf

# (
#     yf.download(tickers=['MSFT', 'AAPL', 'TSLA', 'AMZN', 'NVDA', 'GOOG'])
#     .assign(Date = lambda x: x.index.tz_localize(None))
#     .set_index('Date')
#     .rename_axis(columns=['Variable', 'Ticker'])
#     .to_csv('yahoo.csv')
# )

## Questions

### Question 1

Complete the following calculations:

1. Read the daily price data for the MATANA stocks saved in file `yahoo.csv` and assign them to data frame `yahoo`
1. Calculate the maximum Sharpe Ratio portfolio weights assign them to array `weights` 
    1. Use the daily returns on the MATANA stocks from 2011 through 2022 (inclusive)
    1. Use a target or benchmark return of 0 (instead of the risk-free rate)
    1. Calculate long-only portfolio weights, but do not allow any portfolio weight to exceed 0.3
1. Calculate rolling Sharpe Ratios for the maximum Sharpe Ratio portfolio and assign them to series `sr_rolling`
    1. Use 252-trading day windows
    1. Require at least 200 trading days to calculate each Sharpe Ratio
    
This question has 6 hidden tests worth 5 points each.

_Points:_ 100

In [4]:
yahoo = pd.read_csv("yahoo.csv", header= [0,1], index_col=[0])
yahoo.tail()

Variable     Adj Close                                                  \
Ticker            AAPL        AMZN        GOOG        MSFT        NVDA   
Date                                                                     
2023-04-04  165.630005  103.949997  105.120003  287.179993  274.529999   
2023-04-05  163.759995  101.099998  104.949997  284.339996  268.809998   
2023-04-06  164.660004  102.059998  108.900002  291.600006  270.369995   
2023-04-10  162.029999  102.169998  106.949997  289.390015  275.790009   
2023-04-11  161.509995  100.480698  106.794998  283.815002  273.739990   

Variable                     Close                                      ...  \
Ticker            TSLA        AAPL        AMZN        GOOG        MSFT  ...   
Date                                                                    ...   
2023-04-04  192.580002  165.630005  103.949997  105.120003  287.179993  ...   
2023-04-05  185.520004  163.759995  101.099998  104.949997  284.339996  ...   
2023-04-06  185.059998  164.660004  102.059998  108.900002  291.600006  ...   
2023-04-10  184.509995  162.029999  102.169998  106.949997  289.390015  ...   
2023-04-11         NaN  161.509995  100.480698  106.794998  283.815002  ...   

Variable          Open                                        Volume  \
Ticker            GOOG        MSFT        NVDA        TSLA      AAPL   
Date                                                                   
2023-04-04  104.839996  287.230011  279.660004  197.320007  46278300   
2023-04-05  106.120003  285.850006  268.290009  190.520004  51511700   
2023-04-06  105.769997  283.209991  265.839996  183.080002  45390100   
2023-04-10  107.389999  289.209991  268.230011  179.940002  47672100   
2023-04-11  106.919998  285.750000  277.234985         NaN  33158525   

Variable                                                                 
Ticker            AMZN        GOOG        MSFT        NVDA         TSLA  
Date                                                                     
2023-04-04  48662500.0  20377200.0  25824300.0  36859200.0  126463800.0  
2023-04-05  45175400.0  21864200.0  22064800.0  51501500.0  133882500.0  
2023-04-06  43808000.0  34684200.0  29770300.0  39765400.0  123857900.0  
2023-04-10  37221500.0  19723200.0  23079400.0  39438000.0  141873200.0  
2023-04-11  47340602.0  13859076.0  20946649.0  24902157.0          NaN  

[5 rows x 36 columns]

In [5]:
returns = yahoo["Adj Close"].pct_change().loc["2011-01":"2023-01"]
returns.tail()

Ticker,AAPL,AMZN,GOOG,MSFT,NVDA,TSLA
Date,,,,,,
2022-12-23,-0.002798,0.017425,0.017562,0.002267,-0.008671,-0.017551
2022-12-27,-0.013878,-0.025924,-0.020933,-0.007414,-0.071353,-0.114089
2022-12-28,-0.030685,-0.014692,-0.016718,-0.010255,-0.006019,0.033089
2022-12-29,0.028324,0.028844,0.028799,0.027630,0.040396,0.080827
2022-12-30,0.002469,-0.002138,-0.002473,-0.004937,0.000753,0.011164


In [6]:
def sharpe(x,r,tgt=0,t=252):
    rp = np.dot(r,x)
    ret = rp-tgt
    return -(np.sqrt(t)*ret.mean()/ret.std())

In [7]:
weight = sco.minimize(
        fun= sharpe,
        x0= np.ones(returns.shape[1])/returns.shape[1],
        args= (returns),
        bounds= [(0,0.3) for _ in returns],
        constraints = (
            {"type": "eq", "fun": lambda x : x.sum() - 1}
        )
    
)      
weights = weight["x"]

In [8]:
sharpe_port = returns.dot(weights)

In [9]:
def sharpe_p (r,tgt=0, t=252):
    er = r-tgt
    return np.sqrt(t)* er.mean()/er.std()

In [10]:
sr_rolling = sharpe_port.rolling(252,200).apply(sharpe_p)

In [11]:
grader.check("q1")

q1 results: All test cases passed!
q1 - 1 message: yahoo has the right shape (+10 pts)
q1 - 2 message: yahoo has the right Adj Close column sums (+10 pts)
q1 - 3 message: weights has the right shape (+10 pts)
q1 - 4 message: weights values fall between 0 and 0.3 (+10 pts)
q1 - 5 message: weights sum to 1 (+10 pts)
q1 - 6 message: sr_rolling has the right shape (+5 pts)
q1 - 7 message: sr_rolling has the right number of non-missing values (+5 pts)
q1 - 8 message: sr_rolling has a positive sum (+5 pts)
q1 - 9 message: sr_rolling has a sum > 500 (+5 pts)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)

Running your submission against local test cases...

